# Minutes Missed Due to Games Missed

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
data_DIR = "../data/boxscores_player/"
data_DIR1 = "../data/injuries/"
data_DIR2 = "../data/all_in_one_metrics/"
box_DIR = "../data/boxscores_team/"

In [ ]:
year = 2023

# Overall Minutes Missed

In [ ]:
cols = ["PLAYER_ID","PLAYER_NAME","TEAM_ID","TEAM_ABBREVIATION","GP","MIN","W","L","W_PCT"]
df0 = pd.read_parquet(data_DIR +f"NBA_Player_BoxScores_Base_{year}"+".parquet", columns=cols)
df0["MIN_Tot"] = df0["GP"]* df0["MIN"]

In [ ]:
season = str(year) + "-" + str(year + 1)[-2:]
stats = leaguedashteamstats.LeagueDashTeamStats(per_mode_detailed="PerGame", season_type_all_star="Regular Season", season = season)
df1_1 = stats.get_data_frames()[0]

In [ ]:
cols = ["TEAM_ID","TEAM_NAME","GP","W","L","W_PCT"]
df1 = df1_1[cols]

In [ ]:
df2 = pd.merge(df0,df1,on="TEAM_ID", suffixes=["_P","_T"])

In [ ]:
df2["W_D"] = df2["W_T"] - df2["W_P"]
df2["L_D"] = df2["L_T"] - df2["L_P"]
df2["W_PCT_D"] = df2["W_PCT_T"] - df2["W_PCT_P"]
df2["GP_D"] = df2["GP_T"] - df2["GP_P"]
df2["MIN_Proj"] = df2["MIN"] * df2["GP_T"]
df2["MIN_Miss"] = df2["MIN_Proj"]  - df2["MIN_Tot"]

In [ ]:
df2.query("MIN_Tot > 50").nsmallest(10,"W_PCT_D")

In [ ]:
df3 = df2.query("MIN_Tot > 50 & MIN > 15")

In [ ]:
df3[["PLAYER_NAME","MIN_Miss"]][df3["TEAM_ABBREVIATION"] == 'ORL'].sort_values("MIN_Miss",ascending=False)

In [ ]:
df3.nlargest(10,"MIN_Miss")

In [ ]:
df4 = df3.groupby("TEAM_NAME")[["MIN_Miss"]].sum().sort_values(by="MIN_Miss" , ascending=False)
df4 = df4.reset_index()
df4.index +=1
df4 = pd.merge(df4,df1[["TEAM_NAME","GP"]],on="TEAM_NAME")
df4["MIN_Miss_PG"] = (df4["MIN_Miss"]/df4["GP"]).round(1)
df4["MIN_Miss_Per"] = (df4["MIN_Miss_PG"]/240.0).round(3)
df4.insert(4,"MIN_Miss",df4.pop("MIN_Miss"))

In [ ]:
df5 = df4.sort_values("MIN_Miss_PG",ascending=False)
df5 = df5.reset_index(drop=True)
df5 = df5.reset_index()
df5["index"] +=1

In [ ]:
df5

In [ ]:
%%R -i df5
library(tidyverse)
library(gt)
df <- df5
df %>% 
  gt()%>%
  tab_header(
    title = md("**Minutes Missed Due to Injury/Suspensions etc. 2023-24**"),
    subtitle = "Only Minutes of Players who have played 50 Min and averaged 15 MPG in the season are considered"  
    ) %>%
    data_color(columns = c(MIN_Miss_Per), palette = c("green", "red")) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(TEAM_NAME))  %>%
    cols_label(
      index = "#",TEAM_NAME = "Team", MIN_Miss_PG = "Min Missed PG", MIN_Miss_Per = "Min Missed %", MIN_Miss = "Min Missed"
    ) %>%
    fmt_percent(
      columns = MIN_Miss_Per,
      decimals = 1
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 18,
        heading.subtitle.font.size = 10,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/minutes_missed.png",zoom=5) 

# Individual Game Minutes Missed

In [ ]:
df11 = pd.read_parquet(data_DIR + "NBA_Player_BoxScores_" + "Indv" + "_" + str(year) + ".parquet")
df11["GAME_DATE"] = pd.to_datetime(df11["GAME_DATE"], format="%Y-%m-%d")

In [ ]:
df12 = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{year}.csv")

In [ ]:
games_list = list(df11["GAME_ID"].unique())

In [ ]:
dfinj = pd.read_parquet(data_DIR1 + f'NBA_prosptran_injuries_{year}.parquet')

In [ ]:
def is_injured(player_dict,dfinj,pId_missed,game_date):
    missed_games = np.array([False] * len(pId_missed))
    for i,pId in enumerate(pId_missed):
        # player = player_dict[pId]
        df_p = dfinj.query(f'playerID == {pId}').reset_index(drop=True)
        if len(df_p) > 0:
            df_p["Comp"] = df_p["Date"] <= game_date
            idxi = df_p[df_p["Comp"]].index
            if len(idxi) > 0:
                idx = idxi[-1]
                missed_game = df_p["Out"].loc[idx]
                missed_games[i] = missed_game
    gp = missed_games*pId_missed
    pId_m = gp[gp !=0 ]
    list(pId_m)
    return pId_m

In [ ]:
dfma = []
for gameId in games_list:
    df11_g = df11.query(f"GAME_ID == '{gameId}'")
    df11_gg = df11_g.groupby(by="TEAM_ID")
    keys = list(df11_gg.groups)
    for key in keys:
        df11_g1 = df11_gg.get_group(key)
        team = df11_g1["TEAM_ID"].unique()[0]
        game_date = df11_g1["GAME_DATE"].unique()[0]
        players = df11_g1["PLAYER_ID"].to_list()
        players_team = df3.query(f"TEAM_ID == {team}")["PLAYER_ID"].to_list()
        pId_missed = np.array(list(set(players_team) - set(players))).astype(int)
        players_missed = is_injured(player_dict,dfinj,pId_missed,game_date)
        # players_missed = pId_missed
        dfm1 = pd.DataFrame({"PLAYER_ID":players_missed})
        dfm1["TEAM_ID"] = team
        dfm1["GAME_ID"] = gameId
        dfma.append(dfm1)
dfm = pd.concat(dfma)
dfm["PLAYER_ID"] = dfm["PLAYER_ID"].astype(int)

In [ ]:
df_R = []

In [ ]:
%%R -o df_R

library(tidyverse)
library(nbastatR)
Sys.setenv("VROOM_CONNECTION_SIZE" = 131072 * 2)

# Advanced Leaderboards

## Single Season

season <- 2024

df_R <- bref_players_stats(
  seasons = season,
  tables = "advanced",
  include_all_nba = FALSE,
  only_totals = FALSE,
  nest_data = FALSE,
  assign_to_environment = TRUE,
  widen_data = TRUE,
  join_data = TRUE,
  return_message = TRUE
)

In [ ]:
df_bpm = df_R[["idPlayerNBA","slugTeamBREF","ratioBPM"]].reset_index(drop=True)
df_bpm["idPlayerNBA"] = df_bpm["idPlayerNBA"].astype(int)
df_bpm.columns = ["PLAYER_ID","TEAM_ABBREVIATION","BPM"]
df_bpm["BPM"] += 2

In [ ]:
dfm_b = pd.merge(dfm,df0[["PLAYER_ID","MIN","PLAYER_NAME","TEAM_ABBREVIATION"]], on="PLAYER_ID")
dfm_b = pd.merge(dfm_b,df_bpm, on=["PLAYER_ID","TEAM_ABBREVIATION"])
dfm_b = dfm_b.sort_values(by=["GAME_ID","TEAM_ID","PLAYER_ID"]).reset_index(drop=True)
dfm_b = dfm_b[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION','PLAYER_ID', 'PLAYER_NAME', 'MIN', 'BPM']]
dfm_b["TEAM_NAME"] = dfm_b["TEAM_ID"].map(teams_dict)
dfm_b["BPM_MIN"] = dfm_b["BPM"] * dfm_b["MIN"]
df_mis = dfm_b.groupby(["GAME_ID","TEAM_ID","TEAM_NAME"])[["BPM","MIN","BPM_MIN"]].sum().reset_index()

In [ ]:
df_mist = df_mis.groupby(["TEAM_ID","TEAM_NAME"])[["BPM","MIN","BPM_MIN"]].sum().reset_index()

In [ ]:
df_mist1 = df_mist.drop(columns=["TEAM_ID","BPM","MIN"]).sort_values("BPM_MIN",ascending=False)
df_mist_exp = pd.merge(df4,df_mist1,on="TEAM_NAME")
df_mist_exp = df_mist_exp.sort_values(by ="BPM_MIN",ascending=False).reset_index(drop=True).reset_index()
df_mist_exp["index"] +=1

In [ ]:
df_mist_exp.head()

In [ ]:
%%R -i df_mist_exp
library(tidyverse)
library(gt)
df <- df_mist_exp
df %>% 
  gt()%>%
  tab_header(
    title = md("**BPM Minutes Missed Due to Injury/Suspensions etc. 2023-24**"),
    subtitle = md("Only Minutes of Players who have played 50 Min and averaged 15 MPG in the season are considered")
    ) %>%
    data_color(columns = c(MIN_Miss_Per), palette = c("green", "red")) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(TEAM_NAME))  %>%
    cols_label(
      index = "#",TEAM_NAME = "Team", MIN_Miss_PG = "Min Missed PG", MIN_Miss_Per = "Min Missed %", MIN_Miss = "Min Missed", BPM_MIN = "BPM Missed"
    ) %>%
    fmt_percent(
      columns = MIN_Miss_Per,
      decimals = 1
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 18,
        heading.subtitle.font.size = 10,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>%
    tab_source_note(
    source_note = "BPM Missed = BPM x Min Missed summed for all players who missed that game")  %>% 
    tab_source_note(
    source_note = "and then summed over all games the team has played")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/BPM_missed_1.png",zoom=5) 

In [ ]:
cols = ["idPlayerNBA","dpm"]
df_dpm = pd.read_csv(data_DIR2+"NBA_DARKO_Processed.csv", usecols=cols)
df_dpm.columns = ["PLAYER_ID","DPM"]

In [ ]:
dfm_d = pd.merge(dfm,df_dpm, on="PLAYER_ID")
dfm_d = dfm_d.sort_values(by=["GAME_ID","TEAM_ID","PLAYER_ID"]).reset_index(drop=True)
dfm_d = pd.merge(dfm_d,df0[['PLAYER_ID', 'PLAYER_NAME', 'MIN']],on="PLAYER_ID")
dfm_d["TEAM_NAME"] = dfm_d["TEAM_ID"].map(teams_dict)
dfm_d["DPM_MIN"] = dfm_d["DPM"] * dfm_d["MIN"]

In [ ]:
# dfm_d[ dfm_d["TEAM_NAME"] == "Philadelphia 76ers"]

In [ ]:
df_misd = dfm_d.groupby(["GAME_ID","TEAM_ID","TEAM_NAME"])[["DPM","MIN","DPM_MIN"]].sum().reset_index()
df_mistd = df_misd.groupby(["TEAM_ID","TEAM_NAME"])[["DPM","MIN","DPM_MIN"]].sum().reset_index()

In [ ]:
df_mist1d = df_mistd.drop(columns=["TEAM_ID","DPM"]).sort_values("DPM_MIN",ascending=False)
df_mist_expd = pd.merge(df4[["TEAM_NAME","GP"]],df_mist1d,on="TEAM_NAME")
df_mist_expd["MIN_Miss_PG"] = (df_mist_expd["MIN"] / df_mist_expd["GP"]).round(1)
df_mist_expd["MIN_Miss_Per"] = (df_mist_expd["MIN_Miss_PG"]/240).round(3)
df_mist_expd["DPM_PG"] = (df_mist_expd["DPM_MIN"] / df_mist_expd["MIN_Miss_PG"] ).round(2)
df_mist_expd["DPM_MIN"] =df_mist_expd["DPM_MIN"].round(1)
df_mist_expd = df_mist_expd[["TEAM_NAME","GP","MIN_Miss_PG","MIN_Miss_Per","MIN","DPM_MIN"]]
df_mist_expd = df_mist_expd.sort_values(by ="DPM_MIN",ascending=False).reset_index(drop=True).reset_index()
df_mist_expd["index"] +=1

In [ ]:
%%R -i df_mist_expd
library(tidyverse)
library(gt)
df <- df_mist_expd
df %>% 
  gt()%>%
  tab_header(
    title = md("**DPM (DARKO) Minutes Missed Due to Injury/Suspensions etc. 2023-24**"),
    subtitle = md("Only Minutes of Players who have played 50 Min and averaged 15 MPG in the season are considered <br />Now Considering Injury Reports, so that incidents like DNP-CPs are excluded")
    ) %>%
    data_color(columns = c(MIN_Miss_Per), palette = c("green", "red")) %>%
    cols_align(align = "center")  %>%
    cols_align(align = "left",columns = c(TEAM_NAME))  %>%
    cols_label(
      index = "#",TEAM_NAME = "Team", MIN_Miss_PG = "Min Missed PG", MIN_Miss_Per = "Min Missed %", MIN = "Min Missed", DPM_MIN = "DPM Missed", 
    ) %>%
    fmt_percent(
      columns = MIN_Miss_Per,
      decimals = 1
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 18,
        heading.subtitle.font.size = 12,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Min Miss PG (Per Game) = Min Missed/GP (Games Played) | Min Miss % = Min Miss PG/240 Min (240=48x5)  ")  %>%
    tab_source_note(
    source_note = "DPM Missed = DPM x Min Missed summed for all players who missed that game and then summed over all games the team has played")  %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) #%>% gtsave("../figs/team_leaders/darko_missed.png",zoom=5) 